# Chapter 9 - Training and Deploying at Scale

## Exercise

1. **What does a SavedModel contain? How do you inspect its content?**
   
   SavedModel berisi graph definition, variable values, assets, signature definitions, dan metadata model.
   
   Untuk inspect: `saved_model_cli show --dir my_model --all` atau `tf.saved_model.load("my_model")`

2. **When should you use TF Serving? What are its main features? What are some tools you can use to deploy it?**
   
   Gunakan TF Serving untuk production serving dengan high performance, model versioning, dan scalability.
   
   Main features: high performance, model versioning, hot-swapping, request batching, monitoring.
   
   Tools: Docker, Kubernetes, Google Cloud AI Platform, AWS SageMaker, Azure ML.

3. **How do you deploy a model across multiple TF Serving instances?**
   
   Gunakan load balancer untuk distribute requests, deploy containers di multiple servers, gunakan Kubernetes untuk orchestration, dan shared storage untuk model files.

4. **When should you use the gRPC API rather than the REST API to query a model served by TF Serving?**
   
   Gunakan gRPC untuk lower latency, higher throughput, high-frequency requests, dan binary data. Gunakan REST untuk debugging, web integration, dan kemudahan penggunaan.

5. **What are the different ways TFLite reduces a model's size to make it run on a mobile or embedded device?**
   
   TFLite mengurangi ukuran melalui quantization (float32 ke int8), pruning, weight clustering, knowledge distillation, dan operator fusion.

6. **What is quantization-aware training, and why would you need it?**
   
   Training dengan simulasi quantization effects untuk mengurangi accuracy loss saat quantization dan menghasilkan model yang lebih robust untuk mobile deployment.

7. **What are model parallelism and data parallelism? Why is the latter generally recommended?**
   
   Model parallelism: model dibagi across devices. Data parallelism: model sama di-copy ke multiple devices.
   
   Data parallelism direkomendasikan karena lebih mudah implement, scaling linear, dan communication overhead yang predictable.

8. **When training a model across multiple servers, what distribution strategies can you use? How do you choose which one to use?**
   
   Strategies: MirroredStrategy (single machine), MultiWorkerMirroredStrategy (multiple machines), TPUStrategy (TPUs), ParameterServerStrategy (very large models).
   
   Pilih berdasarkan hardware setup dan model size requirements.

In [1]:
import sys

assert sys.version_info >= (3, 7)

In [2]:
IS_COLAB = "google.colab" in sys.modules
if IS_COLAB:
    import os
    os.environ["TF_USE_LEGACY_KERAS"] = "1"
    import tf_keras

In [3]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [4]:
import sys
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    %pip install -q -U google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.5 MB/s eta 0:00:00


In [5]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

# Serving a TensorFlow Model

### Exporting SavedModels

In [6]:
from pathlib import Path
import tensorflow as tf

# extra code – load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# extra code – build & train an MNIST model (also handles image preprocessing)
tf.random.set_seed(42)
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1719/1719 [==============================] - 11s 5ms/step - loss: 0.6794 - accuracy: 0.8282 - val_loss: 0.3659 - val_accuracy: 0.9056
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3504 - accuracy: 0.9022 - val_loss: 0.2965 - val_accuracy: 0.9172
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3015 - accuracy: 0.9143 - val_loss: 0.2625 - val_accuracy: 0.9278
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2713 - accuracy: 0.9236 - val_loss: 0.2408 - val_accuracy: 0.9356
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2480 - accuracy: 0.9301 - val_loss: 0.2220 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2290 - accuracy: 0.9357 - val_loss: 0.2086 - val_accuracy: 0.9402
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - 

In [7]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # extra code

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

### Installing and Starting TensorFlow Serving

In [8]:
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    url = "https://storage.googleapis.com/tensorflow-serving-apt"
    src = "stable tensorflow-model-server tensorflow-model-server-universal"
    !echo 'deb {url} {src}' > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
    !apt update -q && apt-get install -y tensorflow-model-server
    %pip install -q -U tensorflow-serving-api

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  2943  100  2943    0     0   8027      0 --:--:-- --:--:-- --:--:--  8019
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,765 kB]
Get:7 https://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,026 B]
Get:8 http:/

In [9]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [10]:
%%bash --bg
tensorflow_model_server \
    --port=8500 \
    --rest_api_port=8501 \
    --model_name=my_mnist_model \
    --model_base_path="${MODEL_DIR}" >my_server.log 2>&1

In [11]:
import time

time.sleep(2)

### Menggunakan REST

In [12]:
import json

X_new = X_test[:3] 
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [13]:
request_json[:100] + "..." + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

In [14]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status() 
response = response.json()

In [15]:
import numpy as np

y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### Menggunakan grpc

In [16]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0] 
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [17]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [18]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [19]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [20]:
# extra code – shows how to avoid using tf.make_ndarray()
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### Deploying versi baru

In [21]:
np.random.seed(42)
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.7648 - accuracy: 0.7945 - val_loss: 0.3583 - val_accuracy: 0.9044
Epoch 2/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3279 - accuracy: 0.9066 - val_loss: 0.2764 - val_accuracy: 0.9232
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2715 - accuracy: 0.9214 - val_loss: 0.2367 - val_accuracy: 0.9334
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2382 - accuracy: 0.9310 - val_loss: 0.2130 - val_accuracy: 0.9404
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2148 - accuracy: 0.9371 - val_loss: 0.1953 - val_accuracy: 0.9434
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1971 - accuracy: 0.9431 - val_loss: 0.1833 - val_accuracy: 0.9458
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.1825 - accuracy: 0.9475 - val_loss: 0.1700 - val_accurac

In [22]:
model_version = "0002"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

In [23]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # extra code

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/keras_metadata.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

In [24]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"

response = requests.post(server_url, data=request_json)
response.raise_for_status()
response = response.json()

In [25]:
response.keys()

dict_keys(['predictions'])

In [26]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

## Deploying TF Lite

In [32]:
converter = tf.lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open("my_converted_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)

In [33]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [34]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [35]:
tflite_model = converter.convert()
with open("my_converted_keras_model.tflite", "wb") as f:
    f.write(tflite_model)

## Menggunakan GPU

In [36]:
physical_gpus = tf.config.list_physical_devices("GPU")
physical_gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]